In [5]:
import torch
import torch.nn.functional as F
import torch.nn as nn

batch_size=3
max_vehicles = 10
max_vulns = 7
vuln_width = 4
vuln_conv = nn.Conv2d(
    in_channels=vuln_width,
    out_channels=5,
    stride=2,
    kernel_size=2
)
x = torch.ones((batch_size, max_vehicles, max_vulns, vuln_width))
print(x.shape, "raw", "NHWC")
x = x.permute((0,3,1,2))
print(x.shape, "permuted", "NCHW")
x = vuln_conv(x)
print(x.shape, "convd")
norm = nn.LazyBatchNorm2d()
x = norm(x)
print(x.shape, "norm")
x = F.relu(x)
print(x.shape, "relu")

torch.Size([3, 10, 7, 4]) raw NHWC
torch.Size([3, 4, 10, 7]) permuted NCHW
torch.Size([3, 5, 5, 3]) convd
torch.Size([3, 5, 5, 3]) norm
torch.Size([3, 5, 5, 3]) relu


c:\Users\TeamD\.conda\envs\subgame\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
import vehicles
import game
import agents
import evaluation
import torch

In [ ]:
engine = evaluation.Evaluator(
    # attacker=agents.PassiveAgent(),
    attacker=agents.BasicAttackerAgent(1),
    defender=agents.BasicDefenderAgent(monitor_limit=1),
    vehicles=vehicles.JsonVehicleProvider("../subgame/python/solutions.json"),
    game_config=game.GameConfig(
        max_vehicles=30,
        cycle_every=3,
        cycle_num=5,
        cycle_allow_platoon=False
    ),
    num_rounds=1000
)
engine.reset()

In [ ]:
engine.game.state.vehicles[0].vulnerabilities[0].as_tensor()

tensor([0.9400, 1.0000, 0.0000, 0.0000])

In [ ]:
batch_size = 3
vulns_quants = []
vehicle_quants = []
for _ in range(batch_size):
    vulns_quant, vehicle_quant = engine.game.state.as_tensors(engine.game_config.max_vehicles, engine.vehicles.max_vulns)
    vulns_quants.append(vulns_quant)
    vehicle_quants.append(vehicle_quant)

vulns_quant = torch.stack(vulns_quants)
vehicle_quant = torch.stack(vehicle_quants)
print(vulns_quant.shape)
print(vehicle_quant.shape)
del vulns_quants
del vehicle_quants

torch.Size([3, 30, 7, 4])
torch.Size([3, 30, 2])


In [ ]:
import reinforcement_learning
agent = reinforcement_learning.RLDefenderAgent(engine.game)
y = agent.policy_net(vulns_quant, vehicle_quant)
print(y[0].shape, y[1].shape)

torch.Size([3, 8, 13, 2]) torch.Size([3, 4, 29])
torch.Size([972])
torch.Size([30]) torch.Size([30])


c:\Users\TeamD\.conda\envs\subgame\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
from torchsummary import summary
summary(agent.policy_net)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            808
├─BatchNorm2d: 1-2                       16
├─Conv1d: 1-3                            20
├─BatchNorm1d: 1-4                       8
├─Linear: 1-5                            2,530
├─Linear: 1-6                            2,530
Total params: 5,912
Trainable params: 5,912
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            808
├─BatchNorm2d: 1-2                       16
├─Conv1d: 1-3                            20
├─BatchNorm1d: 1-4                       8
├─Linear: 1-5                            2,530
├─Linear: 1-6                            2,530
Total params: 5,912
Trainable params: 5,912
Non-trainable params: 0

In [1]:
import torch
x = torch.randn(30)
print(x)
x = x.heaviside(torch.tensor(1.))
print(x)
x = ((x == 1).nonzero().squeeze())
print(x)
print(frozenset(x.numpy()))

tensor([-0.0844,  1.1729,  0.2859, -1.5900, -0.5371,  0.5839, -0.4895,  0.0965,
         0.6435, -0.9579, -1.8145, -0.8042, -2.8693,  2.4766,  0.1637,  0.0035,
         0.2406,  0.4464,  0.4439, -2.5238, -0.3808, -0.5235,  0.6792, -1.4819,
        -2.5158, -0.6121, -0.0912,  0.9419, -0.4343, -0.3146])
tensor([0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.])
tensor([ 1,  2,  5,  7,  8, 13, 14, 15, 16, 17, 18, 22, 27])
frozenset({1, 2, 5, 7, 8, 13, 14, 15, 16, 17, 18, 22, 27})
